In [1]:
import pandas as pd
import numpy as np
import random
np.random.seed(0)

In [2]:
# Get data

df_papers = pd.read_csv('../konok_data/papers_data.csv')
df_papers.loc[df_papers['field_query'] == ' Big Data', 'field_query'] = 'Big Data' # Fixed Big Data spacing issue
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,journal_pages
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,"['JournalArticle', 'Conference']",Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,NaN
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,['JournalArticle'],Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,NaN
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,NaN
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,"['JournalArticle', 'Conference']",How well can NLP models generalize to a variet...,2022,432,NaN,NaN,5085-5109
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,NaN


In [3]:
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   field_query       1200 non-null   object
 1   paperId           1200 non-null   object
 2   title             1200 non-null   object
 3   url               1200 non-null   object
 4   venue             1122 non-null   object
 5   publicationTypes  1034 non-null   object
 6   abstract          892 non-null    object
 7   year              1200 non-null   int64 
 8   citationCount     1200 non-null   int64 
 9   journal_name      1038 non-null   object
 10  journal_volume    776 non-null    object
 11  journal_pages     645 non-null    object
dtypes: int64(2), object(10)
memory usage: 112.6+ KB


In [4]:
df_papers['field_query'].value_counts()

Machine Learning     100
Data Storage         100
Deep Learning        100
Data Management      100
Quantum Computing    100
Big Data             100
Graph Database       100
NLP                  100
Data Querying        100
Data Modeling        100
Data Processing      100
LLM                  100
Name: field_query, dtype: int64

# Authors & Corresponding Author

In [5]:
df_authors = pd.read_csv('../konok_data/authors_data.csv')
df_authors = df_authors.dropna(subset=['authorId'])
df_authors.head()

,paperId,authorId,name,affiliations
0,29ddc1f43f28af7c846515e32cc167bc66886d0c,2815290.0,N. Houlsby,NaN
1,29ddc1f43f28af7c846515e32cc167bc66886d0c,1911881.0,A. Giurgiu,NaN
2,29ddc1f43f28af7c846515e32cc167bc66886d0c,40569328.0,Stanislaw Jastrzebski,NaN
3,29ddc1f43f28af7c846515e32cc167bc66886d0c,68973833.0,Bruna Morrone,NaN
4,29ddc1f43f28af7c846515e32cc167bc66886d0c,51985388.0,Quentin de Laroussilhe,NaN


In [6]:
df_author_paper = df_authors.groupby('paperId')['authorId'].agg(list).reset_index()
df_author_paper.head()

,paperId,authorId
0,0026ea8a0fd31bdc959a4b9ed4d449f3015be8d1,"[71940750.0, 1657644010.0, 48633912.0, 9042677..."
1,003ef1cd670d01af05afa0d3c72d72228f494432,"[145306564.0, 48751979.0, 2682457.0, 215519324..."
2,0084f3cb0a1754272151c5268a783f24bf5676a0,"[2281247548.0, 1754598.0, 9379106.0, 140926783..."
3,0090023afc66cd2741568599057f4e82b566137c,"[51997673.0, 2775559.0, 51884035.0, 1782658.0,..."
4,0095acc4f2c3255cf38fdf844003c97858adb418,"[2138646471.0, 143655216.0, 1764004.0]"


In [7]:
len(df_author_paper)

1156

In [8]:
df_papers = pd.merge(df_papers, df_author_paper, on='paperId', how='left')
df_papers.rename(columns={'authorId': 'authorIds'}, inplace=True)
df_papers = df_papers.dropna(subset=['authorIds'])
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   field_query       1164 non-null   object
 1   paperId           1164 non-null   object
 2   title             1164 non-null   object
 3   url               1164 non-null   object
 4   venue             1091 non-null   object
 5   publicationTypes  1012 non-null   object
 6   abstract          873 non-null    object
 7   year              1164 non-null   int64 
 8   citationCount     1164 non-null   int64 
 9   journal_name      1006 non-null   object
 10  journal_volume    759 non-null    object
 11  journal_pages     633 non-null    object
 12  authorIds         1164 non-null   object
dtypes: int64(2), object(11)
memory usage: 127.3+ KB


In [9]:
# corresponding author is the main author
df_papers['corresponding_author'] = df_papers['authorIds'].apply(lambda x: x[0])
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,journal_pages,authorIds,corresponding_author
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,"['JournalArticle', 'Conference']",Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,NaN,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,['JournalArticle'],Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,NaN,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,NaN,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,"['JournalArticle', 'Conference']",How well can NLP models generalize to a variet...,2022,432,NaN,NaN,5085-5109,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,NaN,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09


# Cleanup Data

Based on this, venue seems useless. Therefore, I will remove these datapoints.Similarily remove other stuff as well. Keep only useful and complete stuff for now.

In [10]:
df_papers = df_papers.dropna(subset=['venue'])
df_papers = df_papers.dropna(subset=['publicationTypes'])
df_papers = df_papers.dropna(subset=['abstract'])
df_papers = df_papers.drop(columns=['journal_pages']) # its not needed

df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           757 non-null    object 
 1   paperId               757 non-null    object 
 2   title                 757 non-null    object 
 3   url                   757 non-null    object 
 4   venue                 757 non-null    object 
 5   publicationTypes      757 non-null    object 
 6   abstract              757 non-null    object 
 7   year                  757 non-null    int64  
 8   citationCount         757 non-null    int64  
 9   journal_name          672 non-null    object 
 10  journal_volume        506 non-null    object 
 11  authorIds             757 non-null    object 
 12  corresponding_author  757 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 82.8+ KB


In [11]:
# Filter rows where 'venue' contains 'Workshop' after filling NaN values
workshop_rows = df_papers[df_papers['venue'].fillna('').str.contains('Workshop')]

# Change the value of 'publicationTypes' to 'Workshop' for the filtered rows
df_papers.loc[workshop_rows.index, 'publicationTypes'] = "['Workshop']"

In [12]:
# Filter rows where 'publicationTypes' contains 'Conference' after filling NaN values
conference_rows = df_papers[df_papers['publicationTypes'].fillna('').str.contains('Conference')]

# Change the value of 'publicationTypes' to 'Conference' for the filtered rows
df_papers.loc[conference_rows.index, 'publicationTypes'] = "['Conference']"

In [13]:
df_papers['publicationTypes'].value_counts()

['JournalArticle']                      366
['Conference']                          183
['JournalArticle', 'Review']            120
['Review']                               35
['Review', 'JournalArticle']             23
['Workshop']                             11
['JournalArticle', 'Book']               10
['Book', 'JournalArticle']                4
['Study', 'JournalArticle']               1
['CaseReport', 'Review']                  1
['JournalArticle', 'Study']               1
['Book', 'JournalArticle', 'Review']      1
['Book']                                  1
Name: publicationTypes, dtype: int64

In [14]:
# Only Keep the three for now

allowed_values = ["['JournalArticle']", "['Conference']", "['Workshop']"]
df_papers = df_papers[df_papers['publicationTypes'].isin(allowed_values)]

df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 1198
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           560 non-null    object 
 1   paperId               560 non-null    object 
 2   title                 560 non-null    object 
 3   url                   560 non-null    object 
 4   venue                 560 non-null    object 
 5   publicationTypes      560 non-null    object 
 6   abstract              560 non-null    object 
 7   year                  560 non-null    int64  
 8   citationCount         560 non-null    int64  
 9   journal_name          477 non-null    object 
 10  journal_volume        361 non-null    object 
 11  authorIds             560 non-null    object 
 12  corresponding_author  560 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 61.2+ KB


In [15]:
df_papers['publicationTypes'] = df_papers['publicationTypes'].str[2:-2]
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,NaN,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09


In [16]:
df_papers['publicationTypes'].value_counts()

JournalArticle    366
Conference        183
Workshop           11
Name: publicationTypes, dtype: int64

In [17]:
df_papers['journal_name']

0                                                   ArXiv
1                                                   ArXiv
2                                                   ArXiv
3                                                     NaN
4                                                   ArXiv
                              ...                        
1191    Proceedings of the 2016 International Conferen...
1194    Proceedings of the 2017 ACM International Conf...
1195                               Nucleic Acids Research
1197                                                  NaN
1198    2017 IEEE 33rd International Conference on Dat...
Name: journal_name, Length: 560, dtype: object

In [18]:
# Working on journal_volume

# Identify rows where 'publicationTypes' is either 'Workshop' or 'Conference'
mask = df_papers['publicationTypes'].isin(['Workshop', 'Conference'])

# Set corresponding values in 'journal_volume' to an empty string
df_papers.loc[mask, 'journal_volume'] = ""

In [19]:
df_papers['journal_volume'].isna().sum()

43

In [20]:
# implies there are NaN values for Journals too. So, we need to remove them.
df_papers = df_papers.dropna(subset=['journal_volume'])
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 0 to 1198
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           517 non-null    object 
 1   paperId               517 non-null    object 
 2   title                 517 non-null    object 
 3   url                   517 non-null    object 
 4   venue                 517 non-null    object 
 5   publicationTypes      517 non-null    object 
 6   abstract              517 non-null    object 
 7   year                  517 non-null    int64  
 8   citationCount         517 non-null    int64  
 9   journal_name          450 non-null    object 
 10  journal_volume        517 non-null    object 
 11  authorIds             517 non-null    object 
 12  corresponding_author  517 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 56.5+ KB


In [21]:
# check if all journals have journal names or not
df_papers.query('publicationTypes == "JournalArticle"')['journal_volume'].isna().sum()

0

Note : Currently the following information is missing 

- Edition (19th etc)
- Venue should be the location
- Journal name would be changed to publication type name (basically the stuff inside venue)

For Evolution part, I have to add the following : 
- Reviewer
- Comments
- Approved or Not

In [22]:
# Add Location

df_cities = pd.read_csv('../aryan_data/europe-capital-cities.csv')
capitals = df_cities['capital'].tolist()
df_papers['location'] = [random.choice(capitals) for _ in range(len(df_papers))]
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Moscow
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra


In [23]:
# random function to select number of reviewers for a publication
df_papers['number_reviewers'] = np.random.choice([2, 3], size=len(df_papers))

In [24]:
# # Function to select reviewer IDs for each row
# def select_reviewers(row):
#     author_ids = row['authorIds']
#     reviewers = []
    
#     # Generate a list of all author IDs from the 'authorIds' column
#     all_author_ids = set(df_papers['authorIds'].explode())
    
#     # Exclude the author IDs of the current row
#     potential_reviewers = list(all_author_ids - set(author_ids))
    
#     # Randomly select reviewers, excluding empty reviewer lists
#     if potential_reviewers:
#         reviewers = random.sample(potential_reviewers, min(3, len(potential_reviewers)))
        
#     if any(reviewer_id in author_ids for reviewer_id in reviewers):
#         print("Warning: Reviewer ID overlaps with Author ID for row with paperId:", row['paperId'])
    
#     return reviewers

# # Create a new column 'reviewers' containing a list of 3 reviewer IDs for each row
# df_papers['reviewers'] = df_papers.apply(select_reviewers, axis=1)

In [25]:
# Function to select reviewer IDs for each row
def select_reviewers(row):
    author_ids = row['authorIds']
    num_reviewers = row['number_reviewers']  # Get the number of reviewers from the 'number_reviewers' column
    reviewers = []
    
    # Generate a list of all author IDs from the 'authorIds' column
    all_author_ids = set(df_papers['authorIds'].explode())
    
    # Exclude the author IDs of the current row
    potential_reviewers = list(all_author_ids - set(author_ids))
    
    # Randomly select reviewers, excluding empty reviewer lists
    if potential_reviewers:
        reviewers = random.sample(potential_reviewers, min(num_reviewers, len(potential_reviewers)))  # Use num_reviewers instead of 3
        
    if any(reviewer_id in author_ids for reviewer_id in reviewers):
        print("Warning: Reviewer ID overlaps with Author ID for row with paperId:", row['paperId'])
    
    return reviewers

# Create a new column 'reviewers' containing a list of reviewer IDs for each row
df_papers['reviewers'] = df_papers.apply(select_reviewers, axis=1)

In [26]:
df_papers[['reviewers', 'authorIds']]

,reviewers,authorIds
0,"[50052368.0, 115300694.0]","[2815290.0, 1911881.0, 40569328.0, 68973833.0,..."
1,"[2151900144.0, 46307329.0, 2542999.0]","[145222654.0, 3439053.0, 1716179427.0, 4005230..."
2,"[38608914.0, 2285816475.0, 2109716565.0]","[2268272.0, 47079359.0, 143753639.0]"
3,"[153475895.0, 1753223.0]","[1705260.0, 1817207.0, 1805993128.0, 215653883..."
4,"[47540245.0, 2113511266.0, 2152050780.0]","[2114887261.0, 50487261.0, 2169159066.0, 20735..."
...,...,...
1191,"[1394550182.0, 31131132.0]","[2149234208.0, 1699363.0, 2186806073.0]"
1194,"[1764236.0, 1723636206.0, 3471557.0]","[1730344.0, 1891554.0, 2128664093.0]"
1195,"[1414195584.0, 2146335468.0]","[2485039.0, 2039003.0, 2804763.0, 6562624.0, 4..."
1197,"[2115338656.0, 2086632521.0]","[40275640.0, 1736279.0]"


In [27]:
# Paper is published or accepted, means that it is approved

df_papers['approved'] = 'Y'

In [28]:
# reviewer comments

df_reviews = pd.read_csv('../aryan_data/reviewer_comments.csv')

In [29]:
df_reviews.head()

,ReviewId,SentenceId,Text,Label1,Label2,set
0,r0,s0,Thanks for all the explanations on my review a...,non_actionable,other,train
1,r0,s1,While I can now clearly see the contributions ...,actionable,shortcoming,train
2,r0,s2,"The new section ""intuitive analysis"" is very n...",non_actionable,agreement,test
3,r0,s3,Quality ====== The approach seems sound but th...,actionable,shortcoming,train
4,r0,s4,The application to learning from (partially ad...,non_actionable,fact,test


In [30]:


# Function to select review texts for each row
def select_review_texts(row):
    num_reviews = row['number_reviewers']
    all_reviews = list(df_reviews['Text'])  # Assuming 'df_review_text' is your DataFrame containing review texts
    return random.sample(all_reviews, num_reviews)


# Create a new column 'review_texts' containing a list of review texts for each row
df_papers['comments'] = df_papers.apply(select_review_texts, axis=1)

In [31]:
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,number_reviewers,reviewers,approved,comments
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ..."
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Moscow,3,"[2151900144.0, 46307329.0, 2542999.0]",Y,[2. The experimental results are fairly weak c...
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna,2,"[153475895.0, 1753223.0]",Y,"[Good contribution, paper needs to be made cle..."
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra,3,"[47540245.0, 2113511266.0, 2152050780.0]",Y,[The particular structural elements also induc...


In [32]:
df_papers['comments'][2]

['1. The authors claimed that this is the first end-to-end trainable hierarchical attention model, but there is a previous work that also addressed the similar task: Seo et al, Progressive Attention Networks for Visual Attribute Prediction, in Arxiv preprint:1606.02393, 2016',
 'on the positive side: The paper is well written, quality and clarity of the work are good.',
 'There are many typos (e.g., "This advantage is also its difficulty", "Much previous work on language modeling has evaluated ", "We focus in on the task", and others) so the writing needs to be significantly improved for it to be a conference paper, preferably with some help from a native English speaker.']

# Edition & Proceedings

Proceedings & Edition is for Conference & Workshop; Journal Name & Volume is for Journal.


In [33]:
# rename venue to Proceedings

In [34]:
df_papers.query('publicationTypes == "JournalArticle"')['venue']

1                   Neural Information Processing Systems
7                                               arXiv.org
17                                              arXiv.org
23      Transactions of the Association for Computatio...
55                      J. Am. Medical Informatics Assoc.
                              ...                        
1178    IEEE Transactions on Knowledge and Data Engine...
1179                    Proceedings of the VLDB Endowment
1185                      Journal of Open Source Software
1189                                        F1000Research
1195                                   Nucleic Acids Res.
Name: venue, Length: 323, dtype: object

In [35]:
df_papers.query('publicationTypes == "JournalArticle"')['journal_name']

1                                                   ArXiv
7                                                   ArXiv
17                                                  ArXiv
23      Transactions of the Association for Computatio...
55      Journal of the American Medical Informatics As...
                              ...                        
1178    IEEE Transactions on Knowledge and Data Engine...
1179                                    Proc. VLDB Endow.
1185                                J. Open Source Softw.
1189                                        F1000Research
1195                               Nucleic Acids Research
Name: journal_name, Length: 323, dtype: object

In [36]:
df_papers.query('publicationTypes == "Workshop"')['venue']

15      First Workshop on Insights from Negative Resul...
28      Workshop on Biomedical Natural Language Proces...
484     2021 IEEE/ACM Second International Workshop on...
526     Alberto Mendelzon Workshop on Foundations of D...
534     IEEE International Symposium on Parallel & Dis...
954      USENIX Workshop on Hot Topics in Cloud Computing
1141    International Workshop on Data Warehousing and...
1154           International Workshop on the Semantic Web
1155           International Workshop on the Semantic Web
1156    Alberto Mendelzon Workshop on Foundations of D...
1170             2015 IEEE Security and Privacy Workshops
Name: venue, dtype: object

In [37]:
def set_proceedings(row):
    if row['publicationTypes'] in ['Conference', 'Workshop']:
        return row['venue']
    else:
        return np.nan

# Applying the function to create a new column 'proceedings'
df_papers['proceedings'] = df_papers.apply(lambda row: set_proceedings(row), axis=1)

In [38]:
df_papers.query('publicationTypes == "Conference"')['proceedings'].isna().sum()

0

In [39]:
df_papers.query('publicationTypes == "Workshop"')['proceedings'].isna().sum()

0

In [40]:
# check years column

df_papers['year'].isna().sum()

0

In [41]:
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,number_reviewers,reviewers,approved,comments,proceedings
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ...",International Conference on Machine Learning
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Moscow,3,"[2151900144.0, 46307329.0, 2542999.0]",Y,[2. The experimental results are fairly weak c...,NaN
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna,2,"[153475895.0, 1753223.0]",Y,"[Good contribution, paper needs to be made cle...",Conference on Empirical Methods in Natural Lan...
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra,3,"[47540245.0, 2113511266.0, 2152050780.0]",Y,[The particular structural elements also induc...,Annual Meeting of the Association for Computat...


In [42]:
# Edition

df_papers['edition'] = df_papers['proceedings'].str.extract(r'(\d+)(?:st|nd|rd|th)')

In [43]:
df_papers['edition'].isna().sum()

510

In [44]:
df_papers.query('edition != edition and proceedings == proceedings')

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,number_reviewers,reviewers,approved,comments,proceedings,edition
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ...",International Conference on Machine Learning,NaN
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...,NaN
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna,2,"[153475895.0, 1753223.0]",Y,"[Good contribution, paper needs to be made cle...",Conference on Empirical Methods in Natural Lan...,NaN
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra,3,"[47540245.0, 2113511266.0, 2152050780.0]",Y,[The particular structural elements also induc...,Annual Meeting of the Association for Computat...,NaN
5,NLP,13c4e5a6122f3fa2663f63e49537091da6532f35,Are NLP Models really able to Solve Simple Mat...,https://www.semanticscholar.org/paper/13c4e5a6...,North American Chapter of the Association for ...,Conference,The problem of designing NLP solvers for math ...,2021,385,NaN,,"[1443788809.0, 92954142.0, 144260125.0]",1.443789e+09,Dublin,3,"[2805081.0, 2146671969.0, 2328103.0]",Y,"[If this is not the case for some reason, more...",North American Chapter of the Association for ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Data Querying,b5270b32a17fcc2e3dc209add6f4e4ba709b7358,Recent Advances in Querying Probabilistic Know...,https://www.semanticscholar.org/paper/b5270b32...,International Joint Conference on Artificial I...,Conference,We give a survey on recent advances at the for...,2018,19,NaN,,"[1784772.0, 49633004.0, 1690572.0]",1.784772e+06,Budapest,3,"[2129460589.0, 10708829.0, 3382568.0]",Y,"[Moreover, I like the qualitative experiment (...",International Joint Conference on Artificial I...,NaN
1191,Data Querying,2e7f532796eed2847d4c19e3cff03756049e81b4,Diversified Top-k Subgraph Querying in a Large...,https://www.semanticscholar.org/paper/2e7f5327...,SIGMOD Conference,Conference,Subgraph querying in a large data graph is int...,2016,44,Proceedings of the 2016 International Conferen...,,"[2149234208.0, 1699363.0, 2186806073.0]",2.149234e+09,Ljubljana,2,"[1394550182.0, 31131132.0]",Y,"[In general, this is an interesting direction ...",SIGMOD Conference,NaN
1194,Data Querying,834fdec542153aae5fe725df801aac87ba5e8f56,Graph Querying Meets HCI: State of the Art and...,https://www.semanticscholar.org/paper/834fdec5...,SIGMOD Conference,Conference,Querying graph databases has emerged as an imp...,2017,20,Proceedings of the 2017 ACM International Conf...,,"[1730344.0, 18915

In [45]:
df_papers['edition'] = df_papers['journal_name'].str.extract(r'(\d+)(?:st|nd|rd|th)')
df_papers['edition'].isna().sum()

477

In [46]:
df_papers.query('edition != edition and proceedings == proceedings')

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,number_reviewers,reviewers,approved,comments,proceedings,edition
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ...",International Conference on Machine Learning,NaN
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...,NaN
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna,2,"[153475895.0, 1753223.0]",Y,"[Good contribution, paper needs to be made cle...",Conference on Empirical Methods in Natural Lan...,NaN
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra,3,"[47540245.0, 2113511266.0, 2152050780.0]",Y,[The particular structural elements also induc...,Annual Meeting of the Association for Computat...,NaN
5,NLP,13c4e5a6122f3fa2663f63e49537091da6532f35,Are NLP Models really able to Solve Simple Mat...,https://www.semanticscholar.org/paper/13c4e5a6...,North American Chapter of the Association for ...,Conference,The problem of designing NLP solvers for math ...,2021,385,NaN,,"[1443788809.0, 92954142.0, 144260125.0]",1.443789e+09,Dublin,3,"[2805081.0, 2146671969.0, 2328103.0]",Y,"[If this is not the case for some reason, more...",North American Chapter of the Association for ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,Data Querying,b145f72d9abc4b2c48a46fd1bdc4476a4b5fa4f8,Expressive Time Series Querying with Hand-Draw...,https://www.semanticscholar.org/paper/b145f72d...,International Conference on Human Factors in C...,Conference,"We present Qetch, a tool where users freely sk...",2018,43,Proceedings of the 2018 CHI Conference on Huma...,,"[40964207.0, 1698925.0]",4.096421e+07,Vilnius,3,"[144132699.0, 47319783.0, 2317297.0]",Y,[There's clear value in having good inductive ...,International Conference on Human Factors in C...,NaN
1187,Data Querying,b5270b32a17fcc2e3dc209add6f4e4ba709b7358,Recent Advances in Querying Probabilistic Know...,https://www.semanticscholar.org/paper/b5270b32...,International Joint Conference on Artificial I...,Conference,We give a survey on recent advances at the for...,2018,19,NaN,,"[1784772.0, 49633004.0, 1690572.0]",1.784772e+06,Budapest,3,"[2129460589.0, 10708829.0, 3382568.0]",Y,"[Moreover, I like the qualitative experiment (...",International Joint Conference on Artificial I...,NaN
1191,Data Querying,2e7f532796eed2847d4c19e3cff03756049e81b4,Diversified Top-k Subgraph Querying in a Large...,https://www.semanticscholar.org/paper/2e7f5327...,SIGMOD Conference,Conference,Subgraph querying in a large data graph is int...,2016,44,Proceedings o

In [47]:
# Use the last 2 digits of the year

# Create a boolean mask to identify rows where 'proceedings' is not NaN but 'edition' is NaN
mask = (df_papers['proceedings'].notna()) & (df_papers['edition'].isna())

# Update 'edition' column for rows satisfying the condition
df_papers.loc[mask, 'edition'] = df_papers.loc[mask, 'year'].astype(str).str[-2:]

In [48]:
df_papers['edition'].isna().sum()

323

In [49]:
df_papers['proceedings'].isna().sum()

323

# Journal Name & Volume

In [50]:
df_papers.query('publicationTypes == "JournalArticle"')['journal_name'].isna().sum()

0

In [51]:
df_papers.query('publicationTypes == "JournalArticle"')['venue'].isna().sum()

0

In [52]:
df_papers.query('publicationTypes == "JournalArticle" and journal_name == " "')

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,number_reviewers,reviewers,approved,comments,proceedings,edition


In [53]:
def set_journalname(row):
    if row['publicationTypes'] == 'JournalArticle':
        return row['venue']
    else:
        return np.nan

# Applying the function to create a new column 'proceedings'
df_papers['journal'] = df_papers.apply(lambda row: set_journalname(row), axis=1)

In [54]:
df_papers.query('publicationTypes == "JournalArticle"')['journal'].isna().sum()

0

In [55]:
df_papers.query('publicationTypes == "JournalArticle"')['journal_volume'].isna().sum()

0

In [56]:
# rename ad remove columns
df_papers.drop(columns=['venue'], inplace=True)
df_papers.drop(columns=['journal_name'], inplace=True)
df_papers.rename(columns={'journal': 'journal_name'}, inplace=True)
df_papers.rename(columns={'location': 'venue'}, inplace=True)

In [57]:
df_papers.head()

,field_query,paperId,title,url,publicationTypes,abstract,year,citationCount,journal_volume,authorIds,corresponding_author,venue,number_reviewers,reviewers,approved,comments,proceedings,edition,journal_name
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ...",International Conference on Machine Learning,19,NaN
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,JournalArticle,Large pre-trained language models have been sh...,2020,1686,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Moscow,3,"[2151900144.0, 46307329.0, 2542999.0]",Y,[2. The experimental results are fairly weak c...,NaN,NaN,Neural Information Processing Systems
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Conference,Recent progress in hardware and methodology fo...,2019,2113,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...,19,NaN
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference,How well can NLP models generalize to a variet...,2022,432,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,Vienna,2,"[153475895.0, 1753223.0]",Y,"[Good contribution, paper needs to be made cle...",Conference on Empirical Methods in Natural Lan...,22,NaN
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Conference,Language models (LMs) are pretrained on divers...,2023,72,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Andorra,3,"[47540245.0, 2113511266.0, 2152050780.0]",Y,[The particular structural elements also induc...,Annual Meeting of the Association for Computat...,23,NaN


In [58]:
# Add keywords

df_keywords = pd.read_csv('../konok_data/keyword_info.csv')
df_keywords.head()

,paperId,keywords
0,29ddc1f43f28af7c846515e32cc167bc66886d0c,"['task', 'parameters', 'transfer', 'nlp']"
1,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,"['models', 'tasks', 'language', 'nlp']"
2,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,"['nlp', 'hardware', 'networks', 'models']"
3,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,"['instructions', 'nlp', 'models', 'tasks']"
4,5471114e37448bea2457b74894b1ecb92bbcfdf6,"['lms', 'biases', 'data', 'nlp']"


In [59]:
len(df_keywords)

524

In [60]:
len(df_papers)

517

In [61]:
df_papers = pd.merge(df_papers, df_keywords, on='paperId', how='inner')

In [62]:
df_papers.head()

,field_query,paperId,title,url,publicationTypes,abstract,year,citationCount,journal_volume,authorIds,corresponding_author,venue,number_reviewers,reviewers,approved,comments,proceedings,edition,journal_name,keywords
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2815290.0,Nicosia,2,"[50052368.0, 115300694.0]",Y,"[In the videos, it seems like the robot could ...",International Conference on Machine Learning,19,NaN,"['task', 'parameters', 'transfer', 'nlp']"
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,JournalArticle,Large pre-trained language models have been sh...,2020,1686,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",145222654.0,Moscow,3,"[2151900144.0, 46307329.0, 2542999.0]",Y,[2. The experimental results are fairly weak c...,NaN,NaN,Neural Information Processing Systems,"['models', 'tasks', 'language', 'nlp']"
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Conference,Recent progress in hardware and methodology fo...,2019,2113,,"[2268272.0, 47079359.0, 143753639.0]",2268272.0,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...,19,NaN,"['nlp', 'hardware', 'networks', 'models']"
3,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Conference,Recent progress in hardware and methodology fo...,2019,2113,,"[2268272.0, 47079359.0, 143753639.0]",2268272.0,Bratislava,3,"[38608914.0, 2285816475.0, 2109716565.0]",Y,[1. The authors claimed that this is the first...,Annual Meeting of the Association for Computat...,19,NaN,"['hardware', 'nlp', 'networks', 'deep learning']"
4,Deep Learning,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Conference,Recent progress in hardware and methodology fo...,2019,2113,,"[2268272.0, 47079359.0, 143753639.0]",2268272.0,Bucharest,2,"[2420527.0, 3062937.0]",Y,[The graphs plot number of noisy labels per cl...,Annual Meeting of the Association for Computat...,19,NaN,"['nlp', 'hardware', 'networks', 'models']"


In [63]:
df_papers.to_csv('../aryan_data/papers_info.csv', index=False)